In [176]:
import pandas as pd
import numpy as np
import os
import re
import json

In [177]:
# input dir
input_dir = '/home/andrew/Downloads/slack_data/Testing Export UL Slack export May 22 2021 - Nov 21 2021'

In [178]:
# get list of files

int_logs = pd.DataFrame()
channels = pd.DataFrame()
users    = pd.DataFrame()
messages = list()

for root, dirs, files in os.walk(input_dir, topdown=True):

  # these are the conf/user info files, specified at root of export
  if root == input_dir:
    for file in files:

      path = os.path.join(root, file)

      # channels.json
      if file == 'channels.json':
        data = json.load(open(path, encoding='utf-8'))
        channels = pd.json_normalize(data)

      if file == 'users.json':
        data = json.load(open(path, encoding='utf-8'))
        users = pd.json_normalize(data)

  # the rest are channels with n files by date
  # read each file and append dataframe to a list for concatenation
  else:
    for file in files:

      path = os.path.join(root, file)

      data = json.load(open(path, encoding='utf-8'))

      df = pd.json_normalize(data)

      # adds a column to get channel name
      df['channel_name'] = os.path.basename(root)

      messages.append(df)

# concat all read messages into one frame
df = pd.concat(messages, ignore_index=True)


In [179]:
# convert timestamps to relativity loadable
def to_rel_datetime(series):
  '''
  Returns datetime from timestamp seconds as MM/DD/YYYY HH:MM:SS (24 hour format)
  Implicitly handles null values in the series.
  
  @params: pandas series of timestamp values
  '''
  return pd.to_datetime(series, unit='s').dt.strftime('%m/%d/%Y %H:%M:%S')

# need to convert these columns into rel readable
ts_cols = ['ts', 'thread_ts']

for col in ts_cols:
  df[col + '_converted'] = to_rel_datetime(df[col])


In [180]:
# need to expand some nested json structures and prefix their column names
# reactions, attachments
def unnest(df, cols):
    '''
    For each passed column in passed dataframe, filters out NaN values
    Explodes the list of dictionaries and creates a new series from the exposed dicts
    Adds a prefix to their column of "<COLUMN_NAME> + '.'" (ex. reactions.user)

    @Params: dataframe, dataframe column(s)

    Returns a dataframe
    '''
    for col in cols:
        nfilter = (df[col].notnull())
        df = df.join(df.loc[nfilter, col].explode(0).apply(pd.Series).add_prefix(str(col) + '.'))
    return df

df = unnest(df, ['reactions', 'attachments'])


In [187]:
# get a list of all users in each channel, active or otherwise.
# these are sitting on the channels table in the 'members' column
channel_members = channels[['name', 'members']].copy()
df = df.merge(channel_members, left_on='channel_name', right_on='name', how='left')

In [199]:
# in the 'text' column of the data, want to replace user ID with real name
pattr = re.compile(r'<@(.*)>')
def repl_user_name(cell, user_table, pattr):
  '''
  Function queries the user table by ID and returns that users profile.realname
  Does so with regular expressions as we want to surgically change text in a string

  @Params: cell to operate on, dataframe of user information, regex pattern to search
  '''

  # take string and place id(s) in variable
  matches = re.findall(pattr, cell)

  for match in matches:
    repl = user_table.loc[user_table['id'] == match, 'profile.real_name'].values[0]
    cell = re.sub(pattr, repl, cell)

  return cell

df['text_changed'] = df['text'].apply(
  lambda cell: repl_user_name(cell, users, pattr)
)

In [201]:
nfe = df[['text', 'text_changed']].copy()

In [ ]:
# create a field of date prepending channel name
# will use to group this into 24/hour periods later